In [41]:
# import necessary modules
import numpy as np
import pandas as pd
from random import random, seed
from math import exp

In [42]:
# Initialize Network
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create network and layers as lists
    network = []
    # Create the hidden and output layer where neurons are saved as dictionaries
    hidden_layer = [{"weights":[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    output_layer = [{"weights":[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

In [43]:
# Calculate neuron activation for an input
def activate_neuron(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [44]:
# transfer neuron
def transfer_neuron_sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation))

In [45]:
# forward propagate inputs to a network output
def forward_propagate(network, row):
    inputs = row
    for i, layer in enumerate(network):
        net_inputs = []
        for j, neuron in enumerate(layer):
            activation = activate_neuron(neuron["weights"], inputs)
            neuron["output"] = transfer_neuron_sigmoid(activation)
            net_inputs.append(neuron["output"])
        inputs = net_inputs
    return inputs        

In [46]:
# Calculate the derivative of an neuron output
def transfer_derivative_sigmoid(output):
    return output * (1.0 - output)

In [47]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in range(len(network)-1, -1, -1):
        layer = network[i]
        errors, error_signals = [],[]
        
        for j, neuron in enumerate(layer):
            if i == len(network) -1:
                error = neuron["output"] - expected[j]
                error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                
            else:
                error = 0
                for k, next_neuron in enumerate(network[i+1]):
                    error += next_neuron["weights"][j] * next_neuron["delta"]
                    error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                    errors.append(error)
                    error_signals.append(error_signal) 
            
            errors.append(error)
            error_signals.append(error_signal) 
            neuron['delta'] = error_signal
        

In [49]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        if i == 0:
            inputs = row[:-1]
        else:
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron["weights"][j] -= l_rate * neuron['delta'] * inputs[j]
                neuron["weights"][-1] -= l_rate * neuron['delta']

In [ ]:
#

In [ ]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv'
dataset = pd.read_csv(url)

wheat_seeds = []
for idx, row in dataset.iterrows():
    row_list = []
    for num in row:
        row_list.append(num)
    wheat_seeds.append(row_list)
    
print(np.shape(wheat_seeds))

wheat_inputs = wheat_seeds[:2]
print(wheat_inputs)

In [ ]:
seed(1)

network = initialize_network(7,2,3)
print(network)

In [ ]:
for row in wheat_inputs:
    for layer in network:
        for neuron in layer:
            inputs = forward_propagate(network, row)            

In [ ]:
for layer in network:
    for neuron in layer:
        print(neuron)
        print()
    print()
print()

In [ ]:
print(forward_propagate(network, wheat_inputs[0]))
print(forward_propagate(network, wheat_inputs[1]))

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))